# Cografi Gorsellestirme ve Interaktif Haritalar
## Turkiye Depremleri 1915-2025

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap, MarkerCluster, GroupedLayerControl
from sklearn.cluster import DBSCAN

plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

In [ ]:
kg = pd.read_excel("../data/cleaned/kaggle_cleaned_magnitude4.xlsx")
kg['Date'] = pd.to_datetime(kg['Date'])
kg['Year'] = kg['Date'].dt.year
kg['Month'] = kg['Date'].dt.month
kg['Decade'] = (kg['Year'] // 10) * 10

merged = pd.read_excel("../data/merged/merged_all_matches.xlsx")
merged['Date'] = pd.to_datetime(merged['Date'])

print(f"Kaggle veri seti: {len(kg)} deprem (M >= 4.0)")
print(f"Merged veri seti: {len(merged)} deprem")

In [ ]:
clustering_data = kg[['Latitude', 'Longitude', 'Magnitude', 'Depth', 'Date', 'Location_clean']].dropna().copy()
geo_data = clustering_data[['Latitude', 'Longitude']].values

eps_km = 50
eps_deg = eps_km / 111

dbscan = DBSCAN(eps=eps_deg, min_samples=10)
clustering_data['Cluster'] = dbscan.fit_predict(geo_data)

print(f"Kume sayisi: {len([c for c in clustering_data['Cluster'].unique() if c != -1])}")

In [ ]:
turkey_center = [39.0, 35.0]

m = folium.Map(
    location=turkey_center,
    zoom_start=6,
    tiles=None
)

folium.TileLayer('cartodbpositron', name='Acik Tema').add_to(m)
folium.TileLayer('cartodbdark_matter', name='Koyu Tema').add_to(m)
folium.TileLayer('OpenStreetMap', name='OpenStreetMap').add_to(m)

In [ ]:
heat_group = folium.FeatureGroup(name='Isi Haritasi (Tum Depremler)', show=True)
heat_data = kg[['Latitude', 'Longitude', 'Magnitude']].values.tolist()
HeatMap(
    heat_data,
    min_opacity=0.3,
    max_zoom=10,
    radius=15,
    blur=10,
    gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'yellow', 0.8: 'orange', 1: 'red'}
).add_to(heat_group)
heat_group.add_to(m)

In [ ]:
marker_group = folium.FeatureGroup(name='Deprem Noktalari (M >= 4.0)', show=False)
marker_cluster = MarkerCluster().add_to(marker_group)

for idx, row in kg.iterrows():
    if row['Magnitude'] >= 7.0:
        color = 'darkred'
    elif row['Magnitude'] >= 6.0:
        color = 'red'
    elif row['Magnitude'] >= 5.0:
        color = 'orange'
    else:
        color = 'yellow'
    
    popup_text = f"""
    <b>Tarih:</b> {row['Date'].strftime('%Y-%m-%d')}<br>
    <b>Buyukluk:</b> {row['Magnitude']}<br>
    <b>Derinlik:</b> {row['Depth']} km<br>
    <b>Konum:</b> {row.get('Location_clean', 'Bilinmiyor')}
    """
    
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=max(3, (row['Magnitude'] - 3) * 2),
        color=color,
        fill=True,
        fillColor=color,
        fillOpacity=0.6,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(marker_cluster)

marker_group.add_to(m)

In [ ]:
major_group = folium.FeatureGroup(name='Buyuk Depremler (M >= 6.0)', show=False)
major = kg[kg['Magnitude'] >= 6.0]

for idx, row in major.iterrows():
    if row['Magnitude'] >= 7.0:
        color = '#8B0000'
        radius = 15
    else:
        color = '#FF0000'
        radius = 10
    
    popup_text = f"""
    <b>Tarih:</b> {row['Date'].strftime('%Y-%m-%d')}<br>
    <b>Buyukluk:</b> {row['Magnitude']}<br>
    <b>Derinlik:</b> {row['Depth']} km<br>
    <b>Konum:</b> {row.get('Location_clean', 'Bilinmiyor')}
    """
    
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=radius,
        color=color,
        fill=True,
        fillColor=color,
        fillOpacity=0.7,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(major_group)

major_group.add_to(m)
print(f"Buyuk deprem sayisi (M >= 6.0): {len(major)}")

In [ ]:
deadly_group = folium.FeatureGroup(name='Olumlu Depremler', show=False)
deadly = merged[merged['Total_Deaths'] > 0].copy()

for idx, row in deadly.iterrows():
    deaths = row['Total_Deaths']
    
    if deaths >= 10000:
        radius = 35
        color = '#4B0082'
    elif deaths >= 1000:
        radius = 25
        color = '#8B0000'
    elif deaths >= 100:
        radius = 18
        color = '#FF0000'
    else:
        radius = 12
        color = '#FF4500'
    
    popup_text = f"""
    <b>Tarih:</b> {row['Date'].strftime('%Y-%m-%d')}<br>
    <b>Buyukluk:</b> {row['Magnitude']}<br>
    <b>Olum Sayisi:</b> {int(deaths):,}<br>
    <b>Konum:</b> {row['EMDAT_Location']}
    """
    
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=radius,
        color=color,
        fill=True,
        fillColor=color,
        fillOpacity=0.6,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(deadly_group)

deadly_group.add_to(m)
print(f"Olumlu deprem sayisi: {len(deadly)}")

In [ ]:
fault_group = folium.FeatureGroup(name='Fay Hatlari', show=False)

naf_coords = [
    [40.8, 26.5], [40.7, 28.0], [40.7, 29.5], [40.5, 31.0],
    [40.4, 32.5], [40.2, 34.0], [39.9, 36.0], [39.7, 38.0],
    [39.5, 40.0], [39.3, 41.5], [39.2, 43.0]
]

eaf_coords = [
    [36.5, 36.0], [37.0, 37.0], [37.5, 38.0], [38.0, 39.0],
    [38.5, 40.0], [39.0, 41.0], [39.3, 41.5]
]

baf_coords = [
    [36.8, 27.5], [37.5, 28.5], [38.0, 29.0], [38.5, 29.5],
    [39.0, 30.0]
]

folium.PolyLine(naf_coords, color='blue', weight=5, opacity=0.8,
    popup='Kuzey Anadolu Fayi (KAF)', tooltip='Kuzey Anadolu Fayi').add_to(fault_group)

folium.PolyLine(eaf_coords, color='green', weight=5, opacity=0.8,
    popup='Dogu Anadolu Fayi (DAF)', tooltip='Dogu Anadolu Fayi').add_to(fault_group)

folium.PolyLine(baf_coords, color='purple', weight=5, opacity=0.8,
    popup='Bati Anadolu Fay Zonu', tooltip='Bati Anadolu Fay Zonu').add_to(fault_group)

fault_group.add_to(m)

In [ ]:
shallow = kg[kg['Depth'] <= 10]
medium = kg[(kg['Depth'] > 10) & (kg['Depth'] <= 30)]
deep = kg[kg['Depth'] > 30]

shallow_group = folium.FeatureGroup(name=f'Sik Depremler (0-10 km) ({len(shallow)})', show=False)
for idx, row in shallow.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=3, color='#FF0000', fill=True, fillColor='#FF0000', fillOpacity=0.5
    ).add_to(shallow_group)
shallow_group.add_to(m)

medium_group = folium.FeatureGroup(name=f'Orta Derinlik (10-30 km) ({len(medium)})', show=False)
for idx, row in medium.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=3, color='#FFA500', fill=True, fillColor='#FFA500', fillOpacity=0.5
    ).add_to(medium_group)
medium_group.add_to(m)

deep_group = folium.FeatureGroup(name=f'Derin Depremler (30+ km) ({len(deep)})', show=False)
for idx, row in deep.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=3, color='#0000FF', fill=True, fillColor='#0000FF', fillOpacity=0.5
    ).add_to(deep_group)
deep_group.add_to(m)

print(f"Sik: {len(shallow)}, Orta: {len(medium)}, Derin: {len(deep)}")

In [ ]:
colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', 
          '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe']

cluster_ids = sorted([c for c in clustering_data['Cluster'].unique() if c != -1])

cluster_names = {
    0: 'Dogu Anadolu (Van, Malatya, Elazig)',
    1: 'Ege-Akdeniz (Mugla, Manisa, Izmir)',
    2: 'Ic Anadolu (Ankara, Kastamonu)',
    3: 'Kafkasya Siniri',
    4: 'Kayseri',
    5: 'Tokat',
    6: 'Suriye Siniri',
    7: 'Bartin',
    8: 'Sivas',
    9: 'Kirsehir'
}

cluster_groups = {}
for cluster_id in cluster_ids:
    cluster_data = clustering_data[clustering_data['Cluster'] == cluster_id]
    color = colors[cluster_id % len(colors)]
    cluster_name = cluster_names.get(cluster_id, f'Kume {cluster_id}')
    
    cluster_group = folium.FeatureGroup(name=f'Kume {cluster_id}: {cluster_name} ({len(cluster_data)})', show=False)
    
    for idx, row in cluster_data.iterrows():
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=4, color=color, fill=True, fillColor=color, fillOpacity=0.6
        ).add_to(cluster_group)
    
    center_lat = cluster_data['Latitude'].mean()
    center_lon = cluster_data['Longitude'].mean()
    folium.Marker(
        location=[center_lat, center_lon],
        popup=f'Kume {cluster_id}: {cluster_name}<br>{len(cluster_data)} deprem',
        icon=folium.DivIcon(
            html=f'<div style="font-size:12pt;color:white;background-color:{color};padding:4px 8px;border-radius:50%;text-align:center;font-weight:bold;">{cluster_id}</div>'
        )
    ).add_to(cluster_group)
    
    cluster_group.add_to(m)
    cluster_groups[cluster_id] = cluster_group

In [ ]:
decades = sorted(kg['Decade'].unique())
decade_colors = plt.cm.viridis(np.linspace(0, 1, len(decades)))

decade_groups = {}
for i, decade in enumerate(decades):
    decade_data = kg[kg['Decade'] == decade]
    color = '#{:02x}{:02x}{:02x}'.format(
        int(decade_colors[i][0]*255),
        int(decade_colors[i][1]*255),
        int(decade_colors[i][2]*255)
    )
    
    decade_group = folium.FeatureGroup(name=f"{decade}'ler ({len(decade_data)} deprem)", show=False)
    
    for idx, row in decade_data.iterrows():
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=3, color=color, fill=True, fillColor=color, fillOpacity=0.5
        ).add_to(decade_group)
    
    decade_group.add_to(m)
    decade_groups[decade] = decade_group

In [ ]:
GroupedLayerControl(
    groups={
        'Derinlige Gore Depremler': [shallow_group, medium_group, deep_group],
        'Kumeler': list(cluster_groups.values()),
        'Yillara Gore Depremler': list(decade_groups.values())
    },
    exclusive_groups=False,
    collapsed=False
).add_to(m)

In [ ]:
legend_html = '''
<div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; 
     background-color: white; padding: 15px; border: 2px solid grey; 
     border-radius: 5px; font-size: 11px; max-height: 350px; overflow-y: auto;">
     
     <p style="margin: 0 0 8px 0;"><strong>HARITA REHBERI</strong></p>
     
     <p style="margin: 5px 0;"><strong>Buyukluk:</strong></p>
     <p style="margin: 2px 0;"><span style="color:#8B0000;">&#9679;</span> M 7.0+</p>
     <p style="margin: 2px 0;"><span style="color:#FF0000;">&#9679;</span> M 6.0-6.9</p>
     <p style="margin: 2px 0;"><span style="color:#FFA500;">&#9679;</span> M 5.0-5.9</p>
     <p style="margin: 2px 0;"><span style="color:#FFFF00;">&#9679;</span> M 4.0-4.9</p>
     
     <p style="margin: 8px 0 5px 0;"><strong>Derinlik:</strong></p>
     <p style="margin: 2px 0;"><span style="color:#FF0000;">&#9679;</span> Sik (0-10 km)</p>
     <p style="margin: 2px 0;"><span style="color:#FFA500;">&#9679;</span> Orta (10-30 km)</p>
     <p style="margin: 2px 0;"><span style="color:#0000FF;">&#9679;</span> Derin (30+ km)</p>
     
     <p style="margin: 8px 0 5px 0;"><strong>Fay Hatlari:</strong></p>
     <p style="margin: 2px 0;"><span style="color:blue;">&#9473;</span> Kuzey Anadolu</p>
     <p style="margin: 2px 0;"><span style="color:green;">&#9473;</span> Dogu Anadolu</p>
     <p style="margin: 2px 0;"><span style="color:purple;">&#9473;</span> Bati Anadolu</p>
     
     <p style="margin: 8px 0 5px 0;"><strong>Olum Sayisi:</strong></p>
     <p style="margin: 2px 0;"><span style="color:#4B0082;">&#9679;</span> 10,000+</p>
     <p style="margin: 2px 0;"><span style="color:#8B0000;">&#9679;</span> 1,000-9,999</p>
     <p style="margin: 2px 0;"><span style="color:#FF0000;">&#9679;</span> 100-999</p>
     <p style="margin: 2px 0;"><span style="color:#FF4500;">&#9679;</span> 1-99</p>
</div>
'''

m.get_root().html.add_child(folium.Element(legend_html))

In [ ]:
folium.LayerControl(collapsed=False).add_to(m)

m.save('../outputs/turkey_earthquakes_multilayer_map.html')
print("Cok katmanli harita kaydedildi: turkey_earthquakes_multilayer_map.html")
m

## 2. Zamansal Analiz Grafikleri

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

yearly = kg.groupby('Year').agg({
    'Magnitude': ['count', 'mean', 'max']
}).reset_index()
yearly.columns = ['Year', 'Count', 'Mean_Mag', 'Max_Mag']

ax1 = axes[0, 0]
ax1.fill_between(yearly['Year'], yearly['Count'], alpha=0.3, color='steelblue')
ax1.plot(yearly['Year'], yearly['Count'], color='steelblue', linewidth=1.5)
ax1.set_xlabel('Yil')
ax1.set_ylabel('Deprem Sayisi')
ax1.set_title('Yillik Deprem Sayisi (M >= 4.0)')
ax1.axhline(yearly['Count'].mean(), color='red', linestyle='--', alpha=0.7, label=f'Ortalama: {yearly["Count"].mean():.1f}')
ax1.legend()

ax2 = axes[0, 1]
ax2.bar(yearly['Year'], yearly['Max_Mag'], color='coral', alpha=0.7, width=1)
ax2.axhline(6.0, color='orange', linestyle='--', label='M 6.0')
ax2.axhline(7.0, color='red', linestyle='--', label='M 7.0')
ax2.set_xlabel('Yil')
ax2.set_ylabel('Maksimum Buyukluk')
ax2.set_title('Yillik Maksimum Deprem Buyuklugu')
ax2.legend()

monthly = kg.groupby('Month').size()
ax3 = axes[1, 0]
month_names = ['Oca', 'Sub', 'Mar', 'Nis', 'May', 'Haz', 'Tem', 'Agu', 'Eyl', 'Eki', 'Kas', 'Ara']
bars = ax3.bar(range(1, 13), monthly.values, color='teal', alpha=0.7)
ax3.set_xticks(range(1, 13))
ax3.set_xticklabels(month_names)
ax3.set_xlabel('Ay')
ax3.set_ylabel('Deprem Sayisi')
ax3.set_title('Aylara Gore Deprem Dagilimi')
ax3.axhline(monthly.mean(), color='red', linestyle='--', label=f'Ortalama: {monthly.mean():.1f}')
ax3.legend()

yearly['Cumulative'] = yearly['Count'].cumsum()
ax4 = axes[1, 1]
ax4.fill_between(yearly['Year'], yearly['Cumulative'], alpha=0.3, color='green')
ax4.plot(yearly['Year'], yearly['Cumulative'], color='green', linewidth=2)
ax4.set_xlabel('Yil')
ax4.set_ylabel('Kumulatif Deprem Sayisi')
ax4.set_title('Kumulatif Deprem Sayisi')

plt.tight_layout()
plt.savefig('../outputs/temporal_analysis.png', dpi=150)
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

yearly['Rolling_5yr'] = yearly['Count'].rolling(window=5, center=True).mean()
yearly['Rolling_10yr'] = yearly['Count'].rolling(window=10, center=True).mean()

ax1 = axes[0]
ax1.bar(yearly['Year'], yearly['Count'], alpha=0.3, color='gray', label='Yillik')
ax1.plot(yearly['Year'], yearly['Rolling_5yr'], color='blue', linewidth=2, label='5 Yillik Ortalama')
ax1.plot(yearly['Year'], yearly['Rolling_10yr'], color='red', linewidth=2, label='10 Yillik Ortalama')
ax1.set_xlabel('Yil')
ax1.set_ylabel('Deprem Sayisi')
ax1.set_title('Deprem Frekansi Trend Analizi')
ax1.legend()

yearly['Rolling_Mag'] = yearly['Mean_Mag'].rolling(window=5, center=True).mean()
ax2 = axes[1]
ax2.scatter(yearly['Year'], yearly['Mean_Mag'], alpha=0.4, color='gray', s=20, label='Yillik Ortalama')
ax2.plot(yearly['Year'], yearly['Rolling_Mag'], color='red', linewidth=2, label='5 Yillik Ortalama')
ax2.set_xlabel('Yil')
ax2.set_ylabel('Ortalama Buyukluk')
ax2.set_title('Ortalama Buyukluk Trend Analizi')
ax2.legend()

plt.tight_layout()
plt.savefig('../outputs/trend_analysis.png', dpi=150)
plt.show()

In [ ]:
pivot = kg.pivot_table(
    values='Magnitude',
    index='Month',
    columns='Decade',
    aggfunc='count',
    fill_value=0
)

plt.figure(figsize=(14, 8))
sns.heatmap(
    pivot,
    cmap='YlOrRd',
    annot=True,
    fmt='d',
    cbar_kws={'label': 'Deprem Sayisi'},
    yticklabels=month_names
)
plt.xlabel('On Yil')
plt.ylabel('Ay')
plt.title('Ay ve On Yila Gore Deprem Dagilimi')
plt.tight_layout()
plt.savefig('../outputs/month_decade_heatmap.png', dpi=150)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 10))

scatter = ax.scatter(
    kg['Longitude'], 
    kg['Latitude'],
    c=kg['Magnitude'],
    cmap='YlOrRd',
    s=kg['Magnitude']**2 * 2,
    alpha=0.5,
    edgecolors='none'
)

cbar = plt.colorbar(scatter, ax=ax, shrink=0.7)
cbar.set_label('Buyukluk', fontsize=12)

ax.set_xlim(25, 45)
ax.set_ylim(35, 43)
ax.set_xlabel('Boylam', fontsize=12)
ax.set_ylabel('Enlem', fontsize=12)
ax.set_title('Turkiye Depremleri (1915-2025, M >= 4.0)', fontsize=14)
ax.grid(True, alpha=0.3)
ax.set_facecolor('#f0f0f0')

plt.tight_layout()
plt.savefig('../outputs/static_earthquake_map.png', dpi=200)
plt.show()

## 3. Ozet

In [ ]:
print("="*60)
print("OLUSTURULAN DOSYALAR")
print("="*60)

outputs = [
    ("turkey_earthquakes_multilayer_map.html", "Cok katmanli interaktif harita"),
    ("temporal_analysis.png", "Zamansal analiz grafikleri"),
    ("trend_analysis.png", "Trend analizi grafikleri"),
    ("month_decade_heatmap.png", "Ay-On yil isi matrisi"),
    ("static_earthquake_map.png", "Statik deprem haritasi")
]

for filename, desc in outputs:
    print(f"  - {filename}: {desc}")

print("\n" + "="*60)
print("DROPDOWN MENU GRUPLARI")
print("="*60)
print("  - Derinlige Gore Depremler (3 katman)")
print("  - Kumeler (10 katman)")
print("  - Yillara Gore Depremler (12 katman)")